# Develop and test an interface to core

In [1]:
%matplotlib inline
from importlib import reload
import main, light_curve, core
import warnings
warnings.filterwarnings('error')

In [2]:
fn='/nfs/farm/g/glast/g/catalog/pointlike/skymodels/P8_10years/uw9011/geminga_weights.pkl'
bw= main.Main('Geminga', interval=1, mjd_range=(0, 55000),  weight_file=fn, verbose=1).binned_weights()

Source Geminga at: (l,b)=(195.134,4.266); ROI radius=5
Loading data from 11 months 
	Selected 140566 photons within 5 deg of  (195.13,4.27)
	Dates: 2008-08-05 04:16 - 2009-06-18 00:25
	MJD  : 54683.2          - 55000.0         
Adding weights from file /nfs/farm/g/glast/g/catalog/pointlike/skymodels/P8_10years/uw9011/geminga_weights.pkl
	28 / 139486 photon pixels are outside weight region
	22058 weights set to NaN
Binning: 317 intervals of 1 days from 54683.00 to 55000.00


In [8]:
import core;  reload(core)

t=58000
cell_pars=dict(tstart=t,tstop=t,exposure=1,photon_times=[],photon_weights=[],
            source_to_background_ratio=SonB)

cells=[]
for c in bw:
    print( c)
    t, exp, w, S, B = [c[x] for x in 't exp w S B'.spit()]
    print(t, exp, S,B)
    if exp==0:
        continue
    cell_pars.update(tstart=t-0.5, tstop=t+0.5, photon_weights=w, 
                     exposure=S, #???
                     source_to_background_ratio=S/B)
    assert len(w)>0
    cells.append(core.Cell(**cell_pars))

NameError: name 't' is not defined

In [ ]:
lc = core.CellsLogLikelihood(cells)

In [4]:
bin

<function bin(number, /)>